*Created: 22 February 2022*<br>
*Last Update: 22 February 2022*<br>
*Author: Yasir Abdur Rohman*<br>
**Property of PT Indonesia Power & Lab Getaran & Diagnosis Mesin Undip**

---

# Pipeline Deployment on Notebook

## 1. Get data from PI System

In [4]:
import urllib3

from osisoft.pidevclub.piwebapi.pi_web_api_client import PIWebApiClient
from osisoft.pidevclub.piwebapi.models import PIAnalysis, PIItemsStreamValues, PIStreamValues, PITimedValue, PIRequest

In [64]:
import numpy as np
import time
from datetime import timedelta, date, datetime

In [126]:
webapi = 'https://pivision.indonesiapower.corp/piwebapi'
#pernah gagal klo pake user domain indonesiapower\
username = 'pisystem'
password = 'Abcd1234!'

def getPIWebApiClient(webapi_url, usernme, psswrd):
    client = PIWebApiClient(webapi, False, 
                            username=usernme, password=psswrd, verifySsl=False)
    return client

client = getPIWebApiClient(webapi, username, password)
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

parent = "af:\\\\pi1\\SLA5."
sensor = ["Generator Gross Capacity",
          "Turbine Lube Oil Cooler Outlet Temperature",
          "Turbine.Bearing Oil Pressure",
          "Turbine.Bearing 1 Metal Temperature",
          "Turbine.Bearing 1 Drain Oil Temperature"]

paths = [parent+sensor[i] for i in range(len(sensor))]
paths

['af:\\\\pi1\\SLA5.Generator Gross Capacity',
 'af:\\\\pi1\\SLA5.Turbine Lube Oil Cooler Outlet Temperature',
 'af:\\\\pi1\\SLA5.Turbine.Bearing Oil Pressure',
 'af:\\\\pi1\\SLA5.Turbine.Bearing 1 Metal Temperature',
 'af:\\\\pi1\\SLA5.Turbine.Bearing 1 Drain Oil Temperature']

In [140]:
# get data and preprocessing
def format_timestamp(utc_datetime):
    now_timestamp = time.time()
    offset = datetime.fromtimestamp(now_timestamp) - datetime.utcfromtimestamp(now_timestamp)
    return utc_datetime + offset

def reduce_columns(df, sensor):
    idx_cols_selected = [i for i in range(df.shape[1]) if i==0 or i%6==0]
    idx_col_timestamp = [1]
    idx = idx_col_timestamp + idx_cols_selected
    
    df = df[df.columns[idx]]
    df.columns = ['date'] + sensor

    # format col timestamp
    result = df.copy()
    result['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%d %H:%M:%S')
    result['date'] = pd.to_datetime(result['date']).apply(format_timestamp)
    return result

def transform_data(df):
    payload = {
        'date': df['date'].strftime("%Y-%m-%d %H:%M:%S"),
        'sensors': df.index.tolist()[1:],
        'actuals': df.tolist()[1:],
    }
    return payload
    
def etl(paths, start_time, end_time, interval):
    # extract
    df = client.data.get_multiple_interpolated_values(paths, start_time=start_time, end_time=end_time, interval=interval)
    df = reduce_columns(df, sensor)
    # transform
    payload = transform_data(df.iloc[0])
    return payload

In [161]:
# example
payload = get_data(paths, '2020-03-31 23:59:00', '2020-03-31 23:59:00', '1m')
payload

{'date': '2020-03-31 23:59:00',
 'sensors': ['Generator Gross Capacity',
  'Turbine Lube Oil Cooler Outlet Temperature',
  'Turbine.Bearing Oil Pressure',
  'Turbine.Bearing 1 Metal Temperature',
  'Turbine.Bearing 1 Drain Oil Temperature'],
 'actuals': [364.3693, 43.81718, 1.50464475, 78.5024948, 53.4734344]}

In [171]:
# looping for each minute
current_time = '2020-03-31 23:59:00'

while True:
    # get data
    payload = get_data(paths, current_time, current_time, '1m')
    print(payload)
    print()
    
    # update current time
    current_time = datetime.strptime(current_time, '%Y-%m-%d %H:%M:%S')
    current_time = current_time + timedelta(minutes=1)
    current_time = current_time.strftime('%Y-%m-%d %H:%M:%S')
    
    time.sleep(3)
    
    if current_time == '2020-04-01 00:01:00':
        break

{'date': '2020-03-31 23:59:00', 'sensors': ['Generator Gross Capacity', 'Turbine Lube Oil Cooler Outlet Temperature', 'Turbine.Bearing Oil Pressure', 'Turbine.Bearing 1 Metal Temperature', 'Turbine.Bearing 1 Drain Oil Temperature'], 'actuals': [364.3693, 43.81718, 1.50464475, 78.5024948, 53.4734344]}

{'date': '2020-04-01 00:00:00', 'sensors': ['Generator Gross Capacity', 'Turbine Lube Oil Cooler Outlet Temperature', 'Turbine.Bearing Oil Pressure', 'Turbine.Bearing 1 Metal Temperature', 'Turbine.Bearing 1 Drain Oil Temperature'], 'actuals': [366.922241, 43.6636047, 1.50428307, 78.58779, 53.4734344]}



## 2. Streaming to API Diagnostic

In [151]:
import boto3
import json

In [173]:
kinesis = boto3.client('kinesis', region_name='ap-southeast-1')
stream_name = "B1-CoolingLoss"

In [175]:
# streaming example
# response = kinesis.put_record(StreamName=stream_name, Data=json.dumps(payload), PartitionKey='partition')
# print(response)

In [176]:
# looping for each minute
# looping for each minute
current_time = '2020-03-31 23:59:00'

while True:
    # get data
    payload = get_data(paths, current_time, current_time, '1m')
    print(payload)
    
    # stream data with kinesis
    response = kinesis.put_record(StreamName=stream_name, Data=json.dumps(payload), PartitionKey='partition')
    print(response)
    
    # update current time
    current_time = datetime.strptime(current_time, '%Y-%m-%d %H:%M:%S')
    current_time = current_time + timedelta(minutes=1)
    current_time = current_time.strftime('%Y-%m-%d %H:%M:%S')
    
    time.sleep(3)
    print()
    
    if current_time == '2020-04-01 00:01:00':
        break

{'date': '2020-03-31 23:59:00', 'sensors': ['Generator Gross Capacity', 'Turbine Lube Oil Cooler Outlet Temperature', 'Turbine.Bearing Oil Pressure', 'Turbine.Bearing 1 Metal Temperature', 'Turbine.Bearing 1 Drain Oil Temperature'], 'actuals': [364.3693, 43.81718, 1.50464475, 78.5024948, 53.4734344]}
{'ShardId': 'shardId-000000000000', 'SequenceNumber': '49627003833910076972687405044451296605315601052695789570', 'ResponseMetadata': {'RequestId': 'ceb80961-a6c3-4ae6-9436-e06214d4a00a', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'ceb80961-a6c3-4ae6-9436-e06214d4a00a', 'x-amz-id-2': '6BTceGFH7OwhLeisX9JewhMCnUSPZi+8v+r7b+OjJZYfMIwsroUdFLIUAWQSnkkOtdRwpk73TVXyB2IKSzsYbf0VwXiJduV2', 'date': 'Tue, 22 Feb 2022 13:36:03 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '110'}, 'RetryAttempts': 0}}

{'date': '2020-04-01 00:00:00', 'sensors': ['Generator Gross Capacity', 'Turbine Lube Oil Cooler Outlet Temperature', 'Turbine.Bearing Oil Pressure', 'Turbine.Bea